# DSFB Assignment 5

In this assignment, you will begin to work with text data and natural language processing. You will analyze aspects of th DonorsChoose.org program. Aspects of this project were first posed as a Kaggle challenge and the data comes from [Kaggle DonorsChoose.org Application Screening challenge](https://www.kaggle.com/c/donorschoose-application-screening/data). We have changed the nature of what you need to do in this assignment (so it does not track what was done in the Kaggle Challenge), but nevertheless using or referring to the Kaggle Challenge repository is not allowed for the assignment.

###  DonorsChoose.org  
  
Founded in 2000 by a high school teacher in the Bronx, DonorsChoose.org empowers public school teachers from across the country to request much-needed materials and experiences for their students. At any given time, there are thousands of classroom requests that can be brought to life with a gift of any amount. DonorsChoose.org receives hundreds of thousands of project proposals each year for classroom projects in need of funding. Right now, a large number of volunteers is needed to manually screen each submission before it's approved to be posted on the DonorsChoose.org website. In this assignment, you will analyze the text of the essays and requirements from each proposal.

<img src="https://cached.imagescaler.hbpl.co.uk/resize/scaleWidth/580/cached.offlinehbpl.hbpl.co.uk/news/NST/C8B9CC1D-03B0-9B80-4CFE78B5B539240F.jpg" width="500" height="500" align="center"/>

Image source: https://cached.imagescaler.hbpl.co.uk/resize/scaleWidth/580/cached.offlinehbpl.hbpl.co.uk/news/NST/C8B9CC1D-03B0-9B80-4CFE78B5B539240F.jpg

### Data

As you will see, this dataset includes many different kinds of features with structured and unstructured data. The dataset consists of application materials (see *application_data.csv*) and resources requested (see *resource_data.csv*). The application materials (see *application_data.csv*) contain the following features.

| Feature name  | Description  |
|----------------|--------------|
| id  | Unique id of the project application    |
| teacher_id    | id of the teacher submitting the application  |
| teacher_prefix    | title of the teacher's name (Ms., Mr., etc.)    |
| school_state    | US state of the teacher's school    |
| project_submitted_datetime    | application submission timestamp    |
| project_grade_category    | school grade levels (PreK-2, 3-5, 6-8, and 9-12)   |
| project_subject_categories   | category of the project (e.g., "Music & The Arts")    |
| project_subject_subcategories    | sub-category of the project (e.g., "Visual Arts")    |
| project_title    | title of the project    |
| project_essay_1    | first essay*   |
| project_essay_2    | second essay*    |
| project_essay_3    | third essay*   |
| project_essay_4    | fourth essay*  |
| project_resource_summary    | summary of the resources needed for the project    |
| teacher_number_of_previously_posted_projects   | number of previously posted applications by the submitting teacher    |
| project_is_approved    | whether DonorsChoose proposal was accepted (0="rejected", 1="accepted"); train.csv only    |


\*Note: Prior to May 17, 2016, the prompts for the essays were as follows:

  * project_essay_1: "Introduce us to your classroom"  

  * project_essay_2: "Tell us more about your students"  

  * project_essay_3: "Describe how your students will use the materials you're requesting"  

  * project_essay_4: "Close by sharing why your project will make a difference"  

Starting on May 17, 2016, the number of essays was reduced from 4 to 2, and the prompts for the first 2 essays were changed to the following:

  * project_essay_1: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."  

  * project_essay_2: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"  

For all projects with project_submitted_datetime of 2016-05-17 and later, the values of project_essay_3 and project_essay_4 will be missing (i.e. NaN).


### Special NLP Libraries

We will use several new libraries for this assignment - so be sure to first install those on your machine by with `pip` in a terminal:

    pip install --user -U nltk
    pip install -U gensim
    pip install -U spacy
    pip install -U pyldavis

## IMPORTS

In [2]:
import importlib
# Standard imports
import numpy  as np
import pandas as pd

import itertools
import random
import math  
import copy

from pprint import pprint  # nicer printing

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Other NLP
import re
import spacy
import nltk
from nltk.corpus import stopwords

# General Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
%matplotlib inline  
import seaborn as sns
sns.set(style="white")

# Special Plotting
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Set the maximum number of rows displayed by pandas
pd.options.display.max_rows = 1000

# Set some CONSTANTS that will be used later
SEED    = 41  # base to generate a random number
SCORE   = 'roc_auc'
FIGSIZE = (16, 10)

# PART 1: Prep

**PROBLEM**: To use a particular model in the `spacy` package, you need to manually download and install that particular model. You will need to run the following code from a terminal: `python -m spacy download en_core_web_sm`. Rather than doing that manually from bash in a separate terminal program, do it inline below using a "magic" command in jupyter. HINT: Use *!* followed by a bash command in a cell to run a bash command.

In [2]:
# Download en_core_web_sm for spacy

!python3 -m spacy download en_core_web_sm

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


**PROBLEM**: To confirm that `spacy` is working (and `en_core_web_sm` is installed on your computer), you should be able to use `spacy.load()` to build a `Language` object to perform some basic nlp. Do that below:

In [3]:
# Test use of spacy by using the spacy.load() function
import spacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')

**PROBLEM**: Use nltk.download() to download a list of raw stopwords. (see NLTK documentation)

In [4]:
# Download NLTK stopwords

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ekaterinakryukova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**PROBLEM**: Use the `stopwords` object from `nltk` to build a list of English stopwords. 

In [5]:
# Get English Stopwords from NLTK
from nltk.corpus import stopwords
stopWords = stopwords.words('english')

In [6]:
print(len(stopWords))

179


**PROBLEM**: Extend your `stop_words` list with some additional stopwords that you believe should be ignored in this particular context.

In [7]:
# Extend the stop word list  

stopWords.extend(['from', 'subject', 're', 'edu', 'use'])

stopWords.extend(['although','engaging','approximately','yet','nan','u','us','would','would','see','big','student'])
print(len(stopWords))

196


### Download the Data

Unlike other projects, this project includes a training set too big for GitHub. Through the terminal lab of Jupyter lab, download the data using the *wget* command, unzip it using the *zip* command and check that it's in the root directory of the project. 

Locations : 

    Applications dataset: https://storage.googleapis.com/dsfm-datasets/text-applications/application_data.csv.zip
    Resources dataset: https://storage.googleapis.com/dsfm-datasets/text-applications/resource_data.csv.zip
    
Hint: Use *wget* and *unzip* commands. Use *!* followed by a bash command in a cell to run a bash command.

**PROBLEM**: wget the data

In [8]:
# wget the data
import wget
wget.download('https://storage.googleapis.com/dsfm-datasets/text-applications/application_data.csv.zip','data')

'data/application_data.csv.zip'

In [9]:
wget.download('https://storage.googleapis.com/dsfm-datasets/text-applications/resource_data.csv.zip','data')

'data/resource_data.csv.zip'

**PROBLEM**: unzip the data

In [10]:
# unzip the data
from zipfile import ZipFile
zip = ZipFile('data/application_data.csv.zip')
zip.extractall('data/application_data')

In [11]:
zip = ZipFile('data/resource_data.csv.zip')
zip.extractall('data/resource_data')


# PART 2: Load Data

**PROBLEM**: Load `application_data.csv` and investigate it a bit.

In [5]:
# Load applications
application_data = pd.read_csv('data/application_data/application_data.csv',parse_dates=['project_submitted_datetime'])
application_data.head(5)


,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


In [6]:
#type of date column
application_data.project_submitted_datetime.dtypes

dtype('<M8[ns]')

In [7]:
#number of nan values
application_data.project_essay_1.isna().sum(),application_data.project_essay_2.isna().sum(),application_data.project_essay_3.isna().sum(),application_data.project_essay_4.isna().sum(),

(0, 0, 175706, 175706)

**PROBLEM**: Load `resource_data.csv` and investigate it a bit.

In [8]:
# Load resources

resource_data = pd.read_csv('data/resource_data/resource_data.csv')
resource_data.head(5)

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95


**PROBLEM**: Some of the essays are NA. Replace NAs with empty strings.

In [9]:
# Replace NA values in essay columns with ''

application_data[['project_essay_3','project_essay_4']]=application_data[['project_essay_3',
                  'project_essay_4']].replace(np.nan, '', regex=True)

In [10]:
#count nan
application_data.project_essay_1.isna().sum(),application_data.project_essay_2.isna().sum(),application_data.project_essay_3.isna().sum(),application_data.project_essay_4.isna().sum(),

(0, 0, 0, 0)

In [11]:
application_data.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved'],
      dtype='object')

**PROBLEM**: To simplify matters, combine all essays into just one feature called "essays"

In [12]:
# Combine essays
application_data['essays']=application_data['project_essay_{}'.format(1)]
for i in range(2,5):
    application_data['essays']+=application_data['project_essay_{}'.format(i)].astype(str)

In [13]:
#get data with all essays
application_data[application_data.project_submitted_datetime<'2016-05-17'].head(1)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,essays
18,p232007,e7a8f866e3174a77ffe37323f032a8ac,Mrs.,FL,2016-04-27 09:58:04,Grades PreK-2,"Applied Learning, Literacy & Language","College & Career Prep, Literature & Writing",Watch Readers Grow!,During our reading workshop students are at da...,My students lack confidence. I have a class wi...,During our reading workshop. We do mini lesson...,Your donations would greatly be a blessing for...,My students need these reading materials to he...,6,1,During our reading workshop students are at da...


In [12]:
#check
application_data['essays'][18]

'During our reading workshop students are at daily 5. My students need activities to help them practice skills in a fun and enjoyable way that is on the level of each child. As the teacher I enjoy conferencing with each student, so the more engaged the students are practicing the skills they.My students lack confidence. I have a class with such great potential. My students need more hands on learning and extra practice to catch and grow a love for reading. My second graders love to learn. We have resources but most are out dated. My students would be so bright if they could only build confidence. I believe in them and now I want to see them bloom.During our reading workshop. We do mini lessons that focus on a skill and then they rotate through daily 5 (centers). These reading activities will help them apply what they learned and reinforce the skills. They rotate through self read, buddy read (carpet) listening, writing,  word work (reading skills). If the class is actively engaged this

In [13]:
for i in range(1,5):
    print(application_data['project_essay_{}'.format(i)][18])

During our reading workshop students are at daily 5. My students need activities to help them practice skills in a fun and enjoyable way that is on the level of each child. As the teacher I enjoy conferencing with each student, so the more engaged the students are practicing the skills they.
My students lack confidence. I have a class with such great potential. My students need more hands on learning and extra practice to catch and grow a love for reading. My second graders love to learn. We have resources but most are out dated. My students would be so bright if they could only build confidence. I believe in them and now I want to see them bloom.
During our reading workshop. We do mini lessons that focus on a skill and then they rotate through daily 5 (centers). These reading activities will help them apply what they learned and reinforce the skills. They rotate through self read, buddy read (carpet) listening, writing,  word work (reading skills). If the class is actively engaged thi

In [14]:

#drop separate columns of essays
for i in range(1,5):
    application_data.drop(columns=['project_essay_{}'.format(i)],inplace=True)

In [15]:
application_data.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'essays'],
      dtype='object')

In [16]:
application_data.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'essays'],
      dtype='object')

In [17]:
resource_data.columns

Index(['id', 'description', 'quantity', 'price'], dtype='object')

**PROBLEM**: Merge the resources and application datasets on the *id* feature.

In [14]:
# Merge two datasets


merged_df=application_data.merge(resource_data, on='id', how='inner')
# Check the data to confirm it worked

merged_df.columns


Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'essays', 'description', 'quantity', 'price'],
      dtype='object')

**PROBLEM**: Keep the following data for additional analysis (the id and the text features): `id`, `school_state`, `project_subject_categories`, `project_subject_subcategories`, `essays`, `description`

In [15]:
FEATURE_NAMES = ['school_state', 'project_subject_categories', 'project_subject_subcategories', 'essays', 'description']

In [16]:
# Keep the Text Featuresss

merged_df_textual=merged_df[['id']+FEATURE_NAMES]

In [17]:
merged_df_textual.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,NV,Literacy & Language,Literacy,Most of my kindergarten students come from low...,Apple - iPod nano� 16GB MP3 Player (8th Genera...
1,p036502,NV,Literacy & Language,Literacy,Most of my kindergarten students come from low...,Apple - iPod nano� 16GB MP3 Player (8th Genera...
2,p039565,GA,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Our elementary school is a culturally rich sch...,Reebok Girls' Fashion Dance Graphic T-Shirt - ...
3,p233823,UT,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Hello;\r\nMy name is Mrs. Brotherton. I teach ...,3doodler Start Full Edu Bundle
4,p185307,NC,Health & Sports,Health & Wellness,My students are the greatest students but are ...,BALL PG 4'' POLY SET OF 6 COLORS


In [19]:
merged_df_textual.head(500000).to_csv('merged_df_textual.csv',index=False)

# PART 3: Preprocess Text

Make an independent copy of the data so we can restart here when testing...

In [31]:
data = copy.copy(merged_df_textual).fillna(' ')  # when "merged" is the pandas dataframe

**PROBLEM**: Define a custom function `clean_punctuation()` to remove some punctuation from your text data. You don't have to do absolutely everything one might want to do - just show that you can do it. Start with each some easy operations with `str.replace()`.

In [32]:
# Define a custom function to clean punctuation from  given text

def clean_punctuation(txt):
    txt=txt.replace('&', ' ')
    txt=txt.replace('.', ' ')
    txt=txt.replace("\\r\\n", " ")
    return txt

**PROBLEM**: Use the `apply()` function from pandas to _apply_ that function down the `essays` column of your data.

In [33]:
# Apply your function to clean the essays column
for feature in ['essays']:
    data[feature]=data[feature].apply(clean_punctuation)
    
    
    
data.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,NV,Literacy & Language,Literacy,Most of my kindergarten students come from low...,Apple - iPod nano� 16GB MP3 Player (8th Genera...
1,p036502,NV,Literacy & Language,Literacy,Most of my kindergarten students come from low...,Apple - iPod nano� 16GB MP3 Player (8th Genera...
2,p039565,GA,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Our elementary school is a culturally rich sch...,Reebok Girls' Fashion Dance Graphic T-Shirt - ...
3,p233823,UT,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Hello; My name is Mrs Brotherton I teach 5th...,3doodler Start Full Edu Bundle
4,p185307,NC,Health & Sports,Health & Wellness,My students are the greatest students but are ...,BALL PG 4'' POLY SET OF 6 COLORS


**PROBLEM**: Define **another** custom function called `clean_re()` to clean your text data using regular expressions. Do at least two "cleanings" (i.e., show that you can use the `re` library).

In [34]:
# Define a custom function to clean some given text
import re

def clean_re(txt):
    p = re.compile(r'[^\w\s]')
    txt=p.sub('', txt)
    
    return txt

In [35]:
# Apply clean_re() to all features
from tqdm import tqdm
for feature in tqdm(FEATURE_NAMES):
    data[feature]=data[feature].fillna('').astype(str).apply(clean_re)
    
    
data.to_csv('clean_re.csv', index=False)    
data.head()


100%|██████████| 5/5 [00:40<00:00,  8.07s/it]


,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,NV,Literacy Language,Literacy,Most of my kindergarten students come from low...,Apple iPod nano 16GB MP3 Player 8th Generatio...
1,p036502,NV,Literacy Language,Literacy,Most of my kindergarten students come from low...,Apple iPod nano 16GB MP3 Player 8th Generatio...
2,p039565,GA,Music The Arts Health Sports,Performing Arts Team Sports,Our elementary school is a culturally rich sch...,Reebok Girls Fashion Dance Graphic TShirt Dd ...
3,p233823,UT,Math Science Literacy Language,Applied Sciences Literature Writing,Hello My name is Mrs Brotherton I teach 5th ...,3doodler Start Full Edu Bundle
4,p185307,NC,Health Sports,Health Wellness,My students are the greatest students but are ...,BALL PG 4 POLY SET OF 6 COLORS


In [36]:
data=pd.read_csv('clean_re.csv')

In [37]:
data['description'].head(10)

0    Apple  iPod nano 16GB MP3 Player 8th Generatio...
1    Apple  iPod nano 16GB MP3 Player 8th Generatio...
2    Reebok Girls Fashion Dance Graphic TShirt  Dd ...
3                       3doodler Start Full Edu Bundle
4                       BALL PG 4 POLY SET OF 6 COLORS
5                     BALL PLAYGROUND POLY 85 SET OF 6
6                            KIT JUMBO GRADESTUFF PACK
7                           PARACHUTE GRIPSTARCHUTE 24
8                           RECESS PACK GRADE K VIOLET
9    Crown Berkey Water Filter With 2 Black and 2 P...
Name: description, dtype: object

In [38]:
data[-10:]

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
1081820,p074761,AZ,Math Science,Applied Sciences Environmental Science,I teach science to eighth graders at a suburba...,Craft E Vinyl 12 x 12 40 Sheets Assorted Glo...
1081821,p074761,AZ,Math Science,Applied Sciences Environmental Science,I teach science to eighth graders at a suburba...,Cricut Explore Air 2 Mint Machine Bundle Hea...
1081822,p074761,AZ,Math Science,Applied Sciences Environmental Science,I teach science to eighth graders at a suburba...,Cricut German Carbide Premium Blade
1081823,p074761,AZ,Math Science,Applied Sciences Environmental Science,I teach science to eighth graders at a suburba...,Cricut Tools Scoring Stylus
1081824,p074761,AZ,Math Science,Applied Sciences Environmental Science,I teach science to eighth graders at a suburba...,DAWEI 200Pcs 2 50mm TopMetalBottom Cover Clip...
1081825,p074761,AZ,Math Science,Applied Sciences Environmental Science,I teach science to eighth graders at a suburba...,Neil Enterprises Pin Back Button Parts for Use...
1081826,p074761,AZ,Math Science,Applied Sciences Environmental Science,I teach science to eighth graders at a suburba...,Permanent Self Adhesive Backed Vinyl Sheets Fo...
1081827,p136737,FL,Literacy Language,Literacy,I work with a group of wonderful second grade ...,IRIS 3Drawer Storage Cart with Organizer Top W...
1081828,p190772,TX,Literacy Language,Literacy,Balanced literacy was a mystery to me until a ...,CE764 ReadyToGo Listening Center for 4
1081829,p190772,TX,Literacy Language,Literacy,Balanced literacy was a mystery to me until a ...,RE209X Leveled LN ReadAlongs


**PROBLEM**: Remove stopwords. (Hint: use stopwords from nltk's `stopwords()` plus any additions you'd like to make. Then, again, define a custom function and then apply it to all features.)

In [39]:
# Define custom function to remove stopwords
df = copy.copy(data)  
def clean_stopword(txt):
    txt=txt.lower().split()
    filtered_sentence = [w for w in txt if  w  not in  stopWords]  
    filtered_sentence=' '.join(filtered_sentence)
    return filtered_sentence

In [40]:
# Apply function to remove stopwords  
from tqdm import tqdm
for feature in tqdm(FEATURE_NAMES):
    df[feature]=df[feature].head(500000).fillna(' ').apply(clean_stopword)
    
    
df.to_csv('clean_stopword.csv', index=False)       
df.head(10)



100%|██████████| 5/5 [03:59<00:00, 47.82s/it]


,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,kindergarten students come lowincome household...,apple ipod nano 16gb mp3 player 8th generation...
1,p036502,nv,literacy language,literacy,kindergarten students come lowincome household...,apple ipod nano 16gb mp3 player 8th generation...
2,p039565,ga,music arts health sports,performing arts team sports,elementary school culturally rich school diver...,reebok girls fashion dance graphic tshirt dd d...
3,p233823,ut,math science literacy language,applied sciences literature writing,hello name mrs brotherton teach 5th grade asce...,3doodler start full bundle
4,p185307,nc,health sports,health wellness,students greatest students socially economical...,ball pg 4 poly set 6 colors
5,p185307,nc,health sports,health wellness,students greatest students socially economical...,ball playground poly 85 set 6
6,p185307,nc,health sports,health wellness,students greatest students socially economical...,kit jumbo gradestuff pack
7,p185307,nc,health sports,health wellness,students greatest students socially economical...,parachute gripstarchute 24
8,p185307,nc,health sports,health wellness,students greatest students socially economical...,recess pack grade k violet
9,p013780,ca,health sports,health wellness,students athletes students interested health p...,crown berkey water filter 2 black 2 pf2 fluori...


In [41]:
df=pd.read_csv('clean_stopword.csv')

In [42]:
df=df.head(500000)

In [43]:
df[-10:]

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
499990,p118681,oh,math science,applied sciences,class 24 curious second graders love exploring...,hot wheels track builder loop
499991,p118681,oh,math science,applied sciences,class 24 curious second graders love exploring...,hot wheels track builder straight track includ...
499992,p118681,oh,math science,applied sciences,class 24 curious second graders love exploring...,motion push pull fast slow amazing science
499993,p118681,oh,math science,applied sciences,class 24 curious second graders love exploring...,move motion forces primary physical science
499994,p243708,ca,math science,mathematics,mistakes learn math classroom students come mi...,google management console license
499995,p243708,ca,math science,mathematics,mistakes learn math classroom students come mi...,hp chromebook 11 g4 education edition 11 6 cel...
499996,p134592,ms,math science,mathematics,belief children desire knowledge academic succ...,amazon fire 7tablet 8gb black
499997,p134592,ms,math science,mathematics,belief children desire knowledge academic succ...,amazon freetime kidproof case amazon fire 7tab...
499998,p134592,ms,math science,mathematics,belief children desire knowledge academic succ...,hh400x math instant learning centers gr 23
499999,p134592,ms,math science,mathematics,belief children desire knowledge academic succ...,jj518 classroom magnetic letters kit


**PROBLEM**: Now use Gensim’s `simple_preprocess()` function to tokenize and clean up your text data. TIP: `simple_preprocess()` returns a list of words, so we want to wrap it with a function that joins the list back together into a string.

In [44]:
# Define custom function to wrap c from gensim
from gensim.utils import simple_preprocess
df2 = copy.copy(df) 
def simple_preprocess_custom(txt):
    txt=simple_preprocess(txt,deacc=True)
    txt=' '.join(txt)
    return txt

In [45]:
# Apply simple_preprocess() to all features

for feature in tqdm(FEATURE_NAMES):
    df2[feature]=df2[feature].fillna('').apply(simple_preprocess_custom)

100%|██████████| 5/5 [03:58<00:00, 47.73s/it]


In [46]:
df2.head()
df2.to_csv('simple_preprocess_custom.csv', index=False)       


In [47]:
df2=pd.read_csv('simple_preprocess_custom.csv')

In [48]:
df2.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,kindergarten students come lowincome household...,apple ipod nano gb mp player th generation lat...
1,p036502,nv,literacy language,literacy,kindergarten students come lowincome household...,apple ipod nano gb mp player th generation lat...
2,p039565,ga,music arts health sports,performing arts team sports,elementary school culturally rich school diver...,reebok girls fashion dance graphic tshirt dd d...
3,p233823,ut,math science literacy language,applied sciences literature writing,hello name mrs brotherton teach th grade ascen...,doodler start full bundle
4,p185307,nc,health sports,health wellness,students greatest students socially economical...,ball pg poly set colors


In [49]:
df2=df2.head(500000)

In [50]:
df2.shape

(500000, 6)

**PROBLEM**: Lemmatize the text. (Hint: Define a custom function and then apply it to all features.)

In [51]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ekaterinakryukova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [52]:
importlib.reload(nltk)
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter

In [53]:
df3=copy.copy(df2)



In [54]:
# Write a lemmatization function based on nltk.stem.WordNetLemmatizer() -didn't complete for night


In [55]:
def get_pos( word ):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]

In [58]:

wnl = WordNetLemmatizer()
w_tokenizer = WhitespaceTokenizer()
def lemmatize_text(text):
    
    #lemmatize = lru_cache(maxsize=50000)(wnl.lemmatize)
    text=[wnl.lemmatize(w,get_pos(w)) for w in w_tokenizer.tokenize(text)]
    return text

In [65]:
# Apply lemmatize_text() to all features  
#As it is impossible to process all data, i will process 500 000
from tqdm import tqdm


from tqdm._tqdm_notebook import tqdm_notebook

for feature in tqdm(['essays', 'description']):
    tqdm_notebook.pandas()
    df3[feature]=df3[feature].fillna(' ').progress_apply(lemmatize_text)



  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [50:36<50:36, 3036.48s/it]


100%|██████████| 2/2 [52:51<00:00, 1585.86s/it]

In [66]:
df3.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,"[kindergarten, student, come, lowincome, house...","[apple, ipod, nano, gb, mp, player, th, genera..."
1,p036502,nv,literacy language,literacy,"[kindergarten, student, come, lowincome, house...","[apple, ipod, nano, gb, mp, player, th, genera..."
2,p039565,ga,music arts health sports,performing arts team sports,"[elementary, school, culturally, rich, school,...","[reebok, girl, fashion, dance, graphic, tshirt..."
3,p233823,ut,math science literacy language,applied sciences literature writing,"[hello, name, mr, brotherton, teach, th, grade...","[doodler, start, full, bundle]"
4,p185307,nc,health sports,health wellness,"[student, greatest, student, socially, economi...","[ball, pg, poly, set, color]"


In [67]:
df3.head()
df3.to_csv('lemmatize_text.csv', index=False)       


In [2]:
df3=pd.read_csv('lemmatize_text.csv')

**PROBLEM**: What happened to the data in the pandas dataframe>

ANSWER: It was converted from long text into a list of individual words.

# PART 4:  Make an LDA topic model for the ESSAYS.

**Note: Part 4 is worth 10 points (the value of 10 individual problems).**

Define an LDA topic model for the `essays`. Compute the "Coherence score." Visually inspect the topic model by inspecting the top keywords from each model. Gensim provides functions for all of these tasks.  

In [3]:

# Create Dictionary
id2word = corpora.Dictionary([d.split() for d in df3['essays']])

# Create Corpus
texts = [d.split() for d in df3['essays']]
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View unique id for each word in the essay
print(corpus[:1])


[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 3), (13, 3), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 2), (41, 1), (42, 2), (43, 1), (44, 2), (45, 2), (46, 3), (47, 1), (48, 1), (49, 1), (50, 1), (51, 3), (52, 1), (53, 1), (54, 1), (55, 3), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 3), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 3), (71, 1), (72, 8), (73, 1), (74, 1), (75, 1), (76, 1), (77, 3), (78, 2), (79, 1), (80, 1), (81, 8), (82, 1), (83, 1), (84, 1), (85, 1), (86, 2), (87, 1), (88, 2), (89, 1), (90, 9), (91, 1), (92, 3), (93, 1)]]


In [17]:
# View word a given id corresponds to for id=0
id2word[0]

"'activity',"

In [18]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[("'activity',", 2),
  ("'alongside',", 1),
  ("'always',", 1),
  ("'around',", 1),
  ("'atrisk',", 1),
  ("'basis']", 1),
  ("'cant',", 1),
  ("'center',", 1),
  ("'come',", 2),
  ("'consider',", 1),
  ("'continue',", 1),
  ("'currently',", 1),
  ("'daily',", 3),
  ("'day',", 3),
  ("'determination',", 1),
  ("'differentiate',", 1),
  ("'distance',", 1),
  ("'dream',", 1),
  ("'english',", 1),
  ("'every',", 1),
  ("'everyone',", 1),
  ("'excite',", 2),
  ("'expose',", 1),
  ("'exposure',", 1),
  ("'favorite',", 1),
  ("'first',", 1),
  ("'flashcard',", 1),
  ("'front',", 1),
  ("'get',", 1),
  ("'give',", 1),
  ("'go',", 2),
  ("'halfway',", 1),
  ("'hard',", 2),
  ("'help',", 1),
  ("'herei',", 1),
  ("'highly',", 1),
  ("'home',", 1),
  ("'house',", 1),
  ("'household',", 1),
  ("'ipod',", 2),
  ("'kid',", 2),
  ("'kindergarten',", 1),
  ("'know',", 2),
  ("'knowledge',", 1),
  ("'language',", 2),
  ("'learn',", 2),
  ("'letter',", 3),
  ("'list',", 1),
  ("'literacy',", 1),
  ("'

In [19]:
#df.project_subject_categories.value_counts()

In [5]:
#if __name__ == '__main__':
#freeze_support()
    ##
    ##z Build Multicore LDA
lda_multicore_model = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics = 5, id2word=id2word)
    # Saving trained model
    #lda_multicore_model.save('LDA_NYT_multicore')


In [6]:
lda_multicore_model.save('LDA_NYT_multicore')

In [4]:
# Loading trained model
lda_multicore_model = gensim.models.ldamodel.LdaModel.load('LDA_NYT_multicore')


In [27]:
df3['essays'].head()

0    ['kindergarten', 'student', 'come', 'lowincome...
1    ['kindergarten', 'student', 'come', 'lowincome...
2    ['elementary', 'school', 'culturally', 'rich',...
3    ['hello', 'name', 'mr', 'brotherton', 'teach',...
4    ['student', 'greatest', 'student', 'socially',...
Name: essays, dtype: object

In [8]:
# Print the Keyword in the 7 topics
print(lda_multicore_model.print_topics())
doc_lda = lda_multicore_model[corpus]

[(0, '0.039*"\'student\'," + 0.023*"\'school\'," + 0.014*"\'learn\'," + 0.008*"\'work\'," + 0.008*"\'help\'," + 0.007*"\'get\'," + 0.007*"\'need\'," + 0.007*"\'play\'," + 0.006*"\'come\'," + 0.006*"\'make\',"'), (1, '0.055*"\'student\'," + 0.019*"\'learn\'," + 0.017*"\'school\'," + 0.009*"\'science\'," + 0.008*"\'help\'," + 0.008*"\'technology\'," + 0.008*"\'many\'," + 0.007*"\'classroom\'," + 0.007*"\'project\'," + 0.006*"\'need\',"'), (2, '0.051*"\'student\'," + 0.051*"\'read\'," + 0.048*"\'book\'," + 0.016*"\'school\'," + 0.014*"\'classroom\'," + 0.013*"\'learn\'," + 0.012*"\'love\'," + 0.011*"\'library\'," + 0.009*"\'level\'," + 0.009*"\'help\',"'), (3, '0.058*"\'student\'," + 0.027*"\'learn\'," + 0.017*"\'classroom\'," + 0.017*"\'school\'," + 0.016*"\'need\'," + 0.012*"\'help\'," + 0.011*"\'work\'," + 0.008*"\'day\'," + 0.007*"\'many\'," + 0.007*"\'seat\',"'), (4, '0.053*"\'student\'," + 0.025*"\'learn\'," + 0.018*"\'school\'," + 0.012*"\'classroom\'," + 0.012*"\'help\'," + 0.011*

In [6]:
# Compute Perplexity
#print('\nPerplexity: ', lda_multicore_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_multicore_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3220930255867976


If you use gensim and the following three variables, then you can visualize topics & keywords with the code below.

    lda_model:    this is an LDA model generated by gensim.models.ldamodel.LdaModel()
    id2word:      this is the dictionary term IDs from corpora.Dictionary()
    corpus:       this is the collection of "documents"


In [7]:
# Visualize topics-keywords
lda_model=lda_multicore_model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.004004 -0.013334       1        1  23.469006
2     -0.086656 -0.007392       2        1  22.467103
3      0.020292 -0.045964       3        1  19.885287
1     -0.002424  0.059144       4        1  19.450486
0      0.064785  0.007546       5        1  14.728118, topic_info=                     Term          Freq         Total Category  logprob  \
510               'book',  8.426460e+05  8.426460e+05  Default  30.0000   
65                'read',  9.804290e+05  9.804290e+05  Default  29.0000   
608            'library',  1.868570e+05  1.868570e+05  Default  28.0000   
425               'seat',  1.000240e+05  1.000240e+05  Default  27.0000   
66              'reader',  1.409390e+05  1.409390e+05  Default  26.0000   
212            'science',  1.617850e+05  1.617850e+05  Default  25.0000   
1222               'art',  1.813820e+05  1.813820e+05  Default  24.0000   
216         'technology',  1.499290e+05  1.499290e+05  Default  23.0000   
419               'play',  1.524480e+05  1.524480e+05  Default  22.0000   
312            'healthy',  6.676200e+04  6.676200e+04  Default  21.0000   
476              'chair',  6.074800e+04  6.074800e+04  Default  20.0000   
576               'game',  1.166000e+05  1.166000e+05  Default  19.0000   
428                'sit',  7.801700e+04  7.801700e+04  Default  18.0000   
361               'math',  2.089850e+05  2.089850e+05  Default  17.0000   
198              'level',  2.455900e+05  2.455900e+05  Default  16.0000   
240          'equipment',  4.598100e+04  4.598100e+04  Default  15.0000   
1203              'text',  7.799400e+04  7.799400e+04  Default  14.0000   
7               'center',  1.103740e+05  1.103740e+05  Default  13.0000   
1600              'team',  4.625300e+04  4.625300e+04  Default  12.0000   
408           'flexible',  3.687900e+04  3.687900e+04  Default  11.0000   
204               'move',  1.110110e+05  1.110110e+05  Default  10.0000   
0             'activity',  1.963740e+05  1.963740e+05  Default   9.0000   
196           'interest',  1.411770e+05  1.411770e+05  Default   8.0000   
45               'learn',  1.354207e+06  1.354207e+06  Default   7.0000   
657             'supply',  2.027810e+05  2.027810e+05  Default   6.0000   
70              'school',  1.222042e+06  1.222042e+06  Default   5.0000   
223             'active',  7.321200e+04  7.321200e+04  Default   4.0000   
1327             'music',  5.524400e+04  5.524400e+04  Default   3.0000   
422             'recess',  3.167400e+04  3.167400e+04  Default   2.0000   
725           'material',  3.277710e+05  3.277710e+05  Default   1.0000   
8045            'mallet',  6.668582e+02  6.690782e+02   Topic1 -10.0752   
23883             'stix',  2.772626e+02  2.783063e+02   Topic1 -10.9528   
11517          'trumpet',  2.074648e+02  2.084184e+02   Topic1 -11.2428   
18279           'sequin',  1.670862e+02  1.680046e+02   Topic1 -11.4593   
8770        'mouthpiece',  6.920144e+02  6.958625e+02   Topic1 -10.0382   
16053           'sorter',  1.633433e+02  1.643835e+02   Topic1 -11.4819   
2016            'warhol',  1.782920e+02  1.794431e+02   Topic1 -11.3944   
8769          'ligature',  1.994805e+02  2.008016e+02   Topic1 -11.2821   
20334             'tuba',  1.229603e+02  1.237760e+02   Topic1 -11.7659   
8883         'caldecott',  3.378487e+02  3.400999e+02   Topic1 -10.7552   
5415        'leapreader',  2.027874e+02  2.041573e+02   Topic1 -11.2656   
9295             'cameo',  3.595481e+02  3.620112e+02   Topic1 -10.6929   
19287            'count']  1.574538e+02  1.585447e+02   Topic1 -11.5187   
28744            'heidi',  1.982126e+02  1.996554e+02   Topic1 -11.2885   
13687            ['band',  1.373048e+02  1.384465e+02   Topic1 -11.6556   
14575          'butcher',  2.525657e+02  2.547260e+02   Topic1 -11.0461   
15151       'percussive',  9.835254e+01  9.924371e+01 

# PART 5:  Make an LDA topic model for the DESCRIPTIONS.

**Note: Part 5 is worth 5 points (the value of 5 individual problems).**

Using the same K (and any other hyperparameters from Part 4), recompute a model for Descriptions. Compare the two sets of results. Do they vary? How? Why? Explain what you find. 

In [17]:
df3['description'].head()

0    ['apple', 'ipod', 'nano', 'gb', 'mp', 'player'...
1    ['apple', 'ipod', 'nano', 'gb', 'mp', 'player'...
2    ['reebok', 'girl', 'fashion', 'dance', 'graphi...
3               ['doodler', 'start', 'full', 'bundle']
4               ['ball', 'pg', 'poly', 'set', 'color']
Name: description, dtype: object

In [18]:

# Create Dictionary
id2word_description = corpora.Dictionary([d.split() for d in df3['description']])

# Create Corpus
texts_description = [d.split() for d in df3['description']]
# Term Document Frequency
corpus_description = [id2word_description.doc2bow(text) for text in texts_description]

# View unique id for each word in the essay
print(corpus_description[:1])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]


In [19]:
# 
import time
start_time = time.time()
##
## Build Multicore LDA
lda_multicore_model_d = gensim.models.ldamulticore.LdaMulticore(corpus_description, num_topics = 5, id2word=id2word_description)
# Saving trained model
lda_multicore_model_d.save('LDA_NYT_multicore_d')
# Loading trained model
lda_multicore_model_d = gensim.models.ldamodel.LdaModel.load('LDA_NYT_multicore_d')
## Print time taken to train the model
print("--- %s seconds ---" % (time.time() - start_time))




--- 284.29154419898987 seconds ---


In [20]:
# Print the Keyword in the 7 topics
print(lda_multicore_model_d.print_topics())
doc_lda_d = lda_multicore_model_d[corpus_description]

[(0, '0.026*"\'gb\'," + 0.012*"\'chromebook\'," + 0.010*"\'memory\'," + 0.009*"\'celeron\'," + 0.009*"\'set\']" + 0.008*"\'black\']" + 0.008*"\'intel\'," + 0.007*"\'inch\'," + 0.007*"\'ram\'," + 0.007*"\'flash\',"'), (1, '0.019*"\'kid\'," + 0.016*"\'color\'," + 0.014*"\'pack\'," + 0.013*"\'chair\'," + 0.013*"\'ball\'," + 0.012*"\'pack\']" + 0.010*"\'inch\'," + 0.010*"\'paper\'," + 0.008*"\'black\'," + 0.007*"\'balance\',"'), (2, '0.029*"\'set\']" + 0.024*"\'book\'," + 0.014*"\'set\'," + 0.011*"\'level\'," + 0.010*"\'book\']" + 0.008*"\'read\'," + 0.006*"\'write\'," + 0.006*"\'paper\'," + 0.006*"\'case\'," + 0.006*"\'bk\',"'), (3, '0.013*"\'pack\']" + 0.007*"\'oz\'," + 0.007*"\'color\'," + 0.007*"\'min\'," + 0.007*"\'set\'," + 0.006*"\'pack\'," + 0.006*"\'subscription\'," + 0.006*"\'set\']" + 0.006*"\'assort\'," + 0.005*"\'bag\',"'), (4, '0.024*"\'ipad\'," + 0.018*"\'mini\'," + 0.017*"\'gb\'," + 0.015*"\'set\']" + 0.014*"[\'apple\'," + 0.012*"\'wifi\'," + 0.009*"\'color\'," + 0.009*"\'m

In [23]:
# Compute Perplexity
#print('\nPerplexity: ', lda_multicore_model_d.log_perplexity(corpus_description))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_multicore_model_d, texts=texts_description, dictionary=id2word_description, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.30504606628457304


In [24]:
# Visualize topics-keywords
lda_model=lda_multicore_model_d
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_description, id2word_description)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.178330 -0.023968       1        1  23.104788
3     -0.106225 -0.094465       2        1  20.763776
1     -0.217625  0.008937       3        1  20.575759
4      0.102864 -0.140943       4        1  17.961972
0      0.042656  0.250439       5        1  17.593705, topic_info=                   Term          Freq         Total Category  logprob  loglift
1                 'gb',  26343.000000  26343.000000  Default  30.0000  30.0000
284             'book',  20511.000000  20511.000000  Default  29.0000  29.0000
99              'ipad',  16318.000000  16318.000000  Default  28.0000  28.0000
288             'mini',  14256.000000  14256.000000  Default  27.0000  27.0000
10            ['apple',   8169.000000   8169.000000  Default  26.0000  26.0000
32               'set']  40058.000000  40058.000000  Default  25.0000  25.0000
656       'chromebook',   7126.000000   7126.000000  Default  24.0000  24.0000
194             'wifi',   7840.000000   7840.000000  Default  23.0000  23.0000
285            'level',   8934.000000   8934.000000  Default  22.0000  22.0000
183             'ball',  12302.000000  12302.000000  Default  21.0000  21.0000
73             'chair',  12474.000000  12474.000000  Default  20.0000  20.0000
995           'memory',   5837.000000   5837.000000  Default  19.0000  19.0000
54               'kid',  21625.000000  21625.000000  Default  18.0000  18.0000
655          'celeron',   5485.000000   5485.000000  Default  17.0000  17.0000
127             'book']   8075.000000   8075.000000  Default  16.0000  16.0000
458            'paper',  11970.000000  11970.000000  Default  15.0000  15.0000
40              'pack',  17783.000000  17783.000000  Default  14.0000  14.0000
659            'intel',   4756.000000   4756.000000  Default  13.0000  13.0000
61           'balance',   7693.000000   7693.000000  Default  12.0000  12.0000
85            'wobble',   7643.000000   7643.000000  Default  11.0000  11.0000
432              'min',   4611.000000   4611.000000  Default  10.0000  10.0000
146            'space',   4218.000000   4218.000000  Default   9.0000   9.0000
422             'gray']   4484.000000   4484.000000  Default   8.0000   8.0000
129             'read',   6879.000000   6879.000000  Default   7.0000   7.0000
564            'write',   5062.000000   5062.000000  Default   6.0000   6.0000
437     'subscription',   4176.000000   4176.000000  Default   5.0000   5.0000
325           ['gaiam',   4097.000000   4097.000000  Default   4.0000   4.0000
69               'sit',   4207.000000   4207.000000  Default   3.0000   3.0000
822               'oz',   7618.000000   7618.000000  Default   2.0000   2.0000
661              'ram',   4241.000000   4241.000000  Default   1.0000   1.0000
589              'pip']   1167.621170   1169.531347   Topic1  -6.4765   1.4635
3686         ['boogie',    262.613863    263.466429   Topic1  -7.9685   1.4619
9743          ['lunch',    225.856218    226.682765   Topic1  -8.1193   1.4615
9375          ['happy',    207.467484    208.308879   Topic1  -8.2042   1.4611
3202          'piggie',   1197.163350   1202.577941   Topic1  -6.4515   1.4606
2976         'nospill',    178.758577    179.586185   Topic1  -8.3531   1.4605
9989          'select']    408.740917    410.633425   Topic1  -7.5261   1.4605
3929         'jackson',    591.838747    594.690750   Topic1  -7.1559   1.4603
3201        'elephant',   1214.271510   1220.174328   Topic1  -6.4373   1.4603
3684         'ewriter',    163.513781    164.355117   Topic1  -8.4423   1.4600
16730    'categorical']    154.316400    155.167207   Topic1  -8.5002   1.4596
3685             'jot',    149.902693    150.742245   Topic1  -8.5292   1.4595
59            ['hokki',   2333.256825   2346.899820   Topic1  -5.7842   1.4593
561           'ampamp',    135.105530    135.932770   Topic1  -8.6331   1.4590
1453    'createachain', 

# PART 6:  Use TextHero and help to improve it.

**Note: This is worth 5 points (the value of 5 individual problems).**

[TextHero](https://texthero.org/) is an opensource project developed by a student from the [TIS Lab of Prof. Younge](www.epfl.ch/labs/tis). Go to the [GIT repository for TextHero](https://github.com/jbesomi/texthero), install the package, review the documentation, and if you are impressed by the package - give it a star and tell others! (Not required)

Once you understand TextHero, then use the package to re-implement major portions of Part 3 of this assignment that you completed above.  

In [39]:

!pip install texthero



  Using cached texthero-1.0.9-py3-none-any.whl (25 kB)


In [1]:
import importlib
# Standard imports
import numpy  as np
import pandas as pd

import itertools
import random
import math  
import copy

from pprint import pprint  # nicer printing

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Other NLP
import re
import spacy
import nltk
from nltk.corpus import stopwords

# General Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
%matplotlib inline  
import seaborn as sns
sns.set(style="white")

# Special Plotting
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# ignore some warnings 
import warnings
warnings.filterwarnings('ignore')

# Set the maximum number of rows displayed by pandas
pd.options.display.max_rows = 1000

# Set some CONSTANTS that will be used later
SEED    = 41  # base to generate a random number
SCORE   = 'roc_auc'
FIGSIZE = (16, 10)

In [26]:
import texthero as hero


In [27]:
data2=pd.read_csv('merged_df_textual.csv')

In [28]:
data2=data2.head(100000)

In [29]:
data2.head(4)

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,NV,Literacy & Language,Literacy,Most of my kindergarten students come from low...,Apple - iPod nano� 16GB MP3 Player (8th Genera...
1,p036502,NV,Literacy & Language,Literacy,Most of my kindergarten students come from low...,Apple - iPod nano� 16GB MP3 Player (8th Genera...
2,p039565,GA,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Our elementary school is a culturally rich sch...,Reebok Girls' Fashion Dance Graphic T-Shirt - ...
3,p233823,UT,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Hello;\r\nMy name is Mrs. Brotherton. I teach ...,3doodler Start Full Edu Bundle


In [30]:
from tqdm import tqdm

from tqdm._tqdm_notebook import tqdm_notebook
for column in tqdm(data2.columns):
    tqdm_notebook.pandas()
    data2[column] = data2[column].pipe(hero.clean)

100%|██████████| 6/6 [00:45<00:00,  7.63s/it]


In [31]:
data2.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,kindergarten students come low income househol...,apple ipod nano 16gb mp3 player 8th generation...
1,p036502,nv,literacy language,literacy,kindergarten students come low income househol...,apple ipod nano 16gb mp3 player 8th generation...
2,p039565,ga,music arts health sports,performing arts team sports,elementary school culturally rich school diver...,reebok girls fashion dance graphic shirt dd da...
3,p233823,ut,math science literacy language,applied sciences literature writing,hello \r\nmy name mrs brotherton teach 5th gra...,3doodler start full edu bundle
4,p185307,nc,health sports,health wellness,students greatest students socially economical...,ball pg poly set colors


In [32]:
for column in tqdm(data2.columns):
    tqdm_notebook.pandas()
    data2[column] = data2[column].pipe(hero.remove_html_tags)

100%|██████████| 6/6 [00:01<00:00,  4.50it/s]


In [33]:
data2.head()

,id,school_state,project_subject_categories,project_subject_subcategories,essays,description
0,p036502,nv,literacy language,literacy,kindergarten students come low income househol...,apple ipod nano 16gb mp3 player 8th generation...
1,p036502,nv,literacy language,literacy,kindergarten students come low income househol...,apple ipod nano 16gb mp3 player 8th generation...
2,p039565,ga,music arts health sports,performing arts team sports,elementary school culturally rich school diver...,reebok girls fashion dance graphic shirt dd da...
3,p233823,ut,math science literacy language,applied sciences literature writing,hello \r\nmy name mrs brotherton teach 5th gra...,3doodler start full edu bundle
4,p185307,nc,health sports,health wellness,students greatest students socially economical...,ball pg poly set colors


In [34]:
#kernel died even though i tried 5 ttimes
"""data2['tfidf'] = (
    hero.tfidf(data2['essays']))"""

"data2['tfidf'] = (\n    hero.tfidf(data2['essays']))"

In [35]:
#kernel died even though i tried 5 ttimes
"""
data2['pca'] = hero.pca(data2['tfidf'])
hero.scatterplot(
    df, 
    col='pca', 
    color='topic', 
    title="PCA BBC Sport news"
)"""

'\ndata2[\'pca\'] = hero.pca(data2[\'tfidf\'])\nhero.scatterplot(\n    df, \n    col=\'pca\', \n    color=\'topic\', \n    title="PCA BBC Sport news"\n)'

**Note: This is worth 5 points (the value of 5 individual problems).**

OpenSourcve packages rely on the community of users to help them grow and improve. Review the [contributing file](https://github.com/jbesomi/texthero/blob/master/CONTRIBUTING.md) for Text Hero and then identify a portion of the documentation that you feel could be improves. Edit/write 1 paragraph of documentation for the package that you believe would improve it. Copy that paragraph in below (into this notebook) so that it can be graded. And - if you think your contribution would truly help the project, please learn how to use git to suggest the change (a pull request) to the manager of the repository (Jonathan Besomi). 

Documentation for nmf

texthero.representation.nmf

nmf(s, n_components=2)

    Perform non-negative matrix factorization.

Find two non-negative matrices (W, H) whose product approximates the non-negative matrix X. 
This factorization can be used for example for dimensionality reduction, source separation or topic extraction.

Parameters

    s: Pandas Series
    n_components: Int. Default is 2.
        Number of components to keep. If n_components is not set or None, all components are kept.

Examples

import texthero as hero

import pandas as pd

s = pd.Series(["Sentence one", "Sentence two"])

custom_pipeline = [preprocessing.lowercase,
                   preprocessing.remove_whitespace]
s= hero.clean(s, custom_pipeline)
s_tf_idf = hero.tfidf(s)
pca=hero.nmf(s_tf_idf)

Documentation for pca

Example:
    
import texthero as hero

import pandas as pd

s = pd.Series(["Sentence one", "Sentence two"])

custom_pipeline = [preprocessing.lowercase,
                   preprocessing.remove_whitespace]
                   
s= hero.clean(s, custom_pipeline)

s_tf_idf = hero.tfidf(s)

pca=hero.pca(s_tf_idf)
